# YuNet Face Detection Model

In [ ]:
import numpy as np
import cv2
import os

from ultralytics import YOLO

In [ ]:
# Create the YuNet face detector
IMG_SIZE = 320
# yunet_path = './yunet_model/face_detection_yunet_2023mar.onnx'
# detector = cv2.FaceDetectorYN_create(yunet_path, "", (IMG_SIZE, IMG_SIZE), score_threshold=0.5)

# Directory where images are stored
# image_dir = 'datasets/Acne_Detection/'
# output_dir = 'datasets/iso_Acne_Detection/'

# Create output directory if it doesn't exist
# os.makedirs(output_dir, exist_ok=True)

In [ ]:
# List all image files in the directory
all_images = [file for file in os.listdir(image_dir) if file.endswith(('.jpg', '.jpeg', 'png'))]

# Calculate the number of images to read (1/8 of total)
# num_images_to_read = max(1, len(all_images) // 8)

# Randomly sample the specified number of images
# sampled_images = random.sample(all_images, num_images_to_read)

# Count missed detections
missed = 0

In [ ]:
# Loop through each image in the directory
for filename in all_images:
    image_path = os.path.join(image_dir, filename)
    image = cv2.imread(image_path)

    # Resize the image to the expected input size
    resized_image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))

    # Detect faces in the image
    faces = detector.detect(resized_image)

    # Check if faces were detected
    if faces[1] is None: 
        missed += 1
    else:
        # Isolate detected faces
        for face in faces[1]:  
            x, y, w, h = int(face[0]), int(face[1]), int(face[2]), int(face[3])  
            x = max(0, x)
            y = max(0, y)
            w = min(w, IMG_SIZE-x)
            h = min(h, IMG_SIZE-y)

            #crop the face 
            cropped_face = resized_image[y:y+h, x:x+w]

            # Save the cropped image
            output_path = os.path.join(output_dir, filename)
            cv2.imwrite(output_path, cropped_face)
  

In [ ]:
sample_size = len(all_images)
print(f"Missed {np.round(missed/sample_size * 100, 2)} % of images.")

# YOLOv8 Object Detection Model

In [ ]:
model = YOLO("models/yolov8n.pt")
model.train(data="datasets/kaggle-acne/data.yaml", epochs=100, imgsz=320, batch=4, conf=0.4, iou=0.3, nms=True, max_det=10, hsv_h=0.03, hsv_s=0.9, hsv_v=0.6)

# Resnet-50 Classification Model

In [ ]:
import tensorflow as tf
from keras.src.applications.resnet import ResNet50
from keras.src.layers import Dense, GlobalAveragePooling2D, Flatten
from keras.src.legacy.preprocessing.image import ImageDataGenerator
from keras.src.optimizers import Adam
from keras.src.models import Model
from keras.src.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau


In [ ]:
IMG_SIZE = 320
IMAGE_SHAPE = [IMG_SIZE, IMG_SIZE, 3]
base_resnet_model = ResNet50(input_shape=IMAGE_SHAPE, weights='imagenet', include_top=False)
# print(base_resnet_model.summary())

In [ ]:
base_resnet_model.trainable = False
# print(base_resnet_model.summary())

In [ ]:
train_path = 'datasets/roboflow_acne/train'
test_path = 'datasets/roboflow_acne/test'
valid_path = 'datasets/roboflow_acne/valid'

x = base_resnet_model.output
x = GlobalAveragePooling2D()(x)
# x = Dense(1024, activation='relu')(x)
x = Dense(4, activation='softmax')(x)

final_resnet_model = Model(inputs = base_resnet_model.input, outputs=x)
print(final_resnet_model.summary())


In [ ]:
# Compile the model
final_resnet_model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Data augmentation
train_datagen = ImageDataGenerator(
  rescale=1.0/255,
  rotation_range=20,
  width_shift_range=0.2,
  height_shift_range=0.2,
  horizontal_flip=True
)

test_datagen = ImageDataGenerator(
  rescale=1.0/255
)

train_set = train_datagen.flow_from_directory(
  train_path,
  target_size=(IMG_SIZE, IMG_SIZE),
  batch_size=16,
  class_mode='categorical'
)

valid_set = test_datagen.flow_from_directory(
  valid_path,
  target_size=(IMG_SIZE, IMG_SIZE),
  batch_size=16,
  class_mode='categorical'
)

test_set = test_datagen.flow_from_directory(
  test_path,
  target_size=(IMG_SIZE, IMG_SIZE),
  batch_size=16,
  class_mode='categorical',
  shuffle=False
)

callbacks = [
  ModelCheckpoint('models/best.keras', verbose=True, save_best_only=True, monitor='val_accuracy'),
  ReduceLROnPlateau(monitor='val_accuracy', patience=10, factor=0.1, verbose=True, min_lr=0.00001),
  EarlyStopping(monitor='val_accuracy', patience=30, verbose=True)
]

final_resnet_model.fit(
  train_set,
  validation_data=valid_set,
  epochs=200,
  callbacks=callbacks,
  steps_per_epoch=len(train_set),
  validation_steps=len(valid_set)
)

In [ ]:
best_resnet_model = tf.keras.models.load_model('models/best.keras')

In [ ]:
test_loss, test_accuracy = best_resnet_model.evaluate(test_set, steps=len(test_set))
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

In [ ]:
train_loss, train_accuracy = best_resnet_model.evaluate(train_set, steps=len(train_set))
print(f"Train Loss: {train_loss}")
print(f"Train Accuracy: {train_accuracy}")

In [ ]:
print(train_set.class_indices)